# Insert Weather data into normalized database

In [1]:
# Import all libraries
import pandas as pd
import sqlite3 as sql
import numpy as np

In [2]:
# Read data from CSV file
weather_data = pd.read_csv("US_WeatherEvents_2016-2019.csv")
weather_data.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [3]:
# Create connection
conn = sql.connect("us_weather_data.db")

In [4]:
# Insert into Type table
types = pd.DataFrame({"Type":weather_data["Type"].unique()})
types.to_sql('Type', conn, if_exists="append", index=False)
types = pd.read_sql('select * from Type', conn).to_dict()["Type"]
print(types)

{0: 'Snow', 1: 'Fog', 2: 'Cold', 3: 'Storm', 4: 'Rain', 5: 'Precipitation', 6: 'Hail'}


In [5]:
# Insert into Severity table
severity = pd.DataFrame({"Severity":weather_data["Severity"].unique()})
severity.to_sql('Severity', conn, if_exists="append", index=False)
severity = pd.read_sql('select * from Severity', conn).to_dict()["Severity"]

In [6]:
# Insert into Location table
airport_codes = list(weather_data["AirportCode"].unique())


locations = weather_data.drop_duplicates('AirportCode').loc[:,["AirportCode", "LocationLat", "LocationLng", "City", "County", "State", "ZipCode"]]

locations.rename(columns = {
    "LocationLat": "Latitude",
    "LocationLng": "Longitude"
}, inplace=True)
print(locations.columns)
locations.to_sql('Location', conn, if_exists="append", index=False)
locations = pd.read_sql('select * from Location', conn).to_dict()
#print(locations)

Index(['AirportCode', 'Latitude', 'Longitude', 'City', 'County', 'State',
       'ZipCode'],
      dtype='object')


In [7]:
# Insert into Event table
events = weather_data.loc[:,["StartTime(UTC)", "EndTime(UTC)", "TimeZone", "AirportCode", "Type", "Severity"]]
events["Type"] = events["Type"].apply(lambda x: list(types.keys())[list(types.values()).index(x)])
events["Severity"] = events["Severity"].apply(lambda x: list(severity.keys())[list(severity.values()).index(x)])

events = events.rename(columns = {
    "StartTime(UTC)": "Start_time",
    "EndTime(UTC)": "End_time",
    "Type": "Type_ID",
    "Severity": "Severity_ID"
})
events.to_sql('Event', conn, if_exists="append", index=False)
events = pd.read_sql('select * from Event', conn)